In [1]:
import sys
sys.path.append('../Util')
import pickle
import os

In [2]:
from IPython.display import Image

In [3]:
from evaluation import correlation, lr, perceptron, get_anova_dims, get_mi_dims
from preparation import prepare_dataset, read_datasets

In [4]:
from we import get_we, initiate_model

In [5]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [6]:
from scipy.stats import f_oneway

In [7]:
from sklearn.metrics import mean_absolute_error, accuracy_score
from sklearn.manifold import TSNE
from sklearn.preprocessing import MinMaxScaler, QuantileTransformer

In [8]:
import warnings
warnings.filterwarnings('ignore')

In [9]:
models = [
    {
        'name': 'flaubert/flaubert_small_cased',
        'label': 'flau_small_c'
    },
    {
    
        'name': 'flaubert/flaubert_base_uncased', 
        'label': 'flau_base_u'

    },
    {
        'name': 'flaubert/flaubert_base_cased',
        'label': 'flau_base_c'
    },
    {
        'name': 'flaubert/flaubert_large_cased',
        'label': 'flau_large_c'
    },
    {
        'name': 'camembert/camembert-base',
        'label': 'cam_base'
    },
    {
        'name': 'xlm-roberta-large',
        'label': 'xlm_large'
    },
    {
    
        'name': 'xlm-roberta-base', 
        'label': 'xlm_base'

    },
    {
        'name': 'bert-base-multilingual-uncased',
        'label': 'bert_base_u'
    },
    {
        'name': 'distilbert-base-multilingual-cased',
        'label': 'distilbert_base'
    },
    {
        'name': 'bert-base-multilingual-cased',
        'label': 'bert_base_c'
    }
    
]

In [10]:
labels = [m['label'] for m in models]

In [11]:
we_with_features = read_datasets(
                            path = '../Data',
                            model_labels = labels,
                            file_name = 'all_unique_pos_we.csv'
                    )

In [12]:
we_with_features[-1].head()

,0,1,2,3,4,5,6,7,8,9,...,764,765,766,767,Number,Gender,Lemma,POS,Tense,Person
Word,,,,,,,,,,,,,,,,,,,,,
2D,0.909603,0.551025,0.436937,0.789782,0.311517,0.417476,0.629341,0.913722,0.349834,0.838988,...,0.285154,0.057887,0.030874,0.310204,invariable,feminine,2D,NOUN,NaN,NaN
3D,0.901779,0.549880,0.427826,0.797102,0.313900,0.419226,0.623478,0.916539,0.355807,0.844714,...,0.287610,0.054251,0.018604,0.305226,invariable,feminine,3D,NOUN,NaN,NaN
aa,0.899570,0.555891,0.418186,0.805754,0.319232,0.412976,0.618457,0.914183,0.367882,0.845985,...,0.287883,0.057476,0.013614,0.290463,invariable,masculine,aa,NOUN,NaN,NaN
abandon,0.898879,0.556306,0.417821,0.805256,0.318463,0.412505,0.618276,0.914983,0.367654,0.846577,...,0.288183,0.057827,0.014050,0.290994,singular,masculine,abandon,NOUN,NaN,NaN
abbaye,0.895888,0.554461,0.420659,0.799339,0.322441,0.418141,0.618820,0.921033,0.367005,0.845904,...,0.293985,0.057312,0.012175,0.298473,singular,feminine,abbaye,NOUN,NaN,NaN


In [13]:
feature_col_count = 6

# Feature to investigate in this notebook
feature = 'POS'

# Nouns

In [14]:
pos = ['ADJ']

Split each model into train and test using k_fold cross validation:

In [15]:
X_adj_train = []
y_adj_train = []

X_adj_test = []
y_adj_test = []

In [16]:
n_folds = 5

In [17]:
for we in we_with_features:
    X, y = prepare_dataset(dataset=we[(we.Gender != 'invariable') & (we.Number != 'invariable')],
                                           feature_col_count=feature_col_count,
                                           feature_name=feature,
                                           normalize=False,
                                           encode=True,
                                           encode_as1='ADJ',
                                           split=False,
                                           balance=True)
    X_trains = []
    y_trains = []
    
    X_tests = []
    y_tests = []
    
    X_folds = np.array_split(X, n_folds)
    y_folds = np.array_split(y, n_folds)
    
    for i in range(n_folds):
        X_trains.append(pd.DataFrame(np.concatenate(X_folds[:i] + X_folds[i+1:])))
        y_trains.append(np.concatenate(y_folds[:i] + y_folds[i+1:]))

        X_folds[i].columns = X_folds[i].columns.map(int)
        X_tests.append(X_folds[i])
        y_tests.append(y_folds[i])
        
    
    X_adj_train.append(X_trains)
    X_adj_test.append(X_tests)
    
    y_adj_train.append(y_trains)
    y_adj_test.append(y_tests)

In [18]:
dataset_sizes = pd.DataFrame(index=labels)
dataset_sizes['1 fold size'] = [len(x[0]) for x in X_adj_test]
dataset_sizes

,1 fold size
flau_small_c,753
flau_base_u,897
flau_base_c,753
flau_large_c,753
cam_base,418
xlm_large,82
xlm_base,82
bert_base_u,166
distilbert_base,106
bert_base_c,106


## Non-independent dims

Calculate non-independendent dimensions for each of _k_ splits.

ANOVA test with p-value < 0.01:

In [19]:
anova_dims = []

In [20]:
for i in range(len(models)):
    anova_dims.append([])
    for j in range(n_folds):
        anova_dims[i].append(get_anova_dims(X_adj_train[i][j], y_adj_train[i][j]))

Get dimensions where MI > 0:

In [21]:
mi_dims = []

In [22]:
for i in range(len(models)):
    mi_dims.append([])
    for j in range(n_folds):
        mi_dims[i].append(get_mi_dims(X_adj_train[i][j], y_adj_train[i][j]))

In [23]:
for i in range(len(models)):
    for j in range(n_folds):
        temp = list(map(lambda x: int(x), mi_dims[i][j]))
        mi_dims[i][j] = temp

Get dimensions that are both found by the ANOVA independency test and MI test:

In [24]:
non_ind_dims = []

In [25]:
for i in range(len(models)):
    non_ind_dims.append([])
    for j in range(n_folds):
        non_ind_dims[i].append(list(set(anova_dims[i][j]).intersection(mi_dims[i][j])))

Stats about the number of dimensions for each model for the 1st split:

In [26]:
non_ind_df = pd.DataFrame(index=labels, columns=['ANOVA', 'MI', 'Total non independent'])

In [27]:
non_ind_df['ANOVA'] = [len(x[0]) for x in anova_dims]
non_ind_df['MI'] = [len(x[0]) for x in mi_dims]
non_ind_df['Total non independent'] = [len(x[0]) for x in non_ind_dims]

In [28]:
non_ind_df

,ANOVA,MI,Total non independent
flau_small_c,383,378,303
flau_base_u,406,664,368
flau_base_c,297,439,197
flau_large_c,714,751,583
cam_base,202,544,135
xlm_large,32,545,25
xlm_base,518,564,379
bert_base_u,2,424,1
distilbert_base,194,524,173
bert_base_c,0,468,0


In [29]:
dims = {}

In [30]:
for i in range(len(models)):
    dims[labels[i]] = {}
    dims[labels[i]]['All dims'] = [X_adj_train[i][0].columns]*n_folds
    dims[labels[i]]['ANOVA'] = anova_dims[i]
    dims[labels[i]]['MI'] = mi_dims[i]
    dims[labels[i]]['All non ind'] = non_ind_dims[i]

## Important dims

We can test different $\alpha$ values: 1%, 5%, 10%, 25%, 50%, 75%.

In [31]:
alphas = [1, 5, 10, 25, 50, 75]

Train Logistic Regression on train set for each model:

In [32]:
lr_res = []

In [33]:
for i in range(len(models)):
    lr_res.append([])
    for j in range(n_folds):
        lr_res[i].append(lr(X_adj_train[i][j], y_adj_train[i][j]))

Train Perceptron:

In [34]:
perceptron_res = []

In [35]:
for i in range(len(models)):
    perceptron_res.append([])
    for j in range(n_folds):
        perceptron_res[i].append(perceptron(X_adj_train[i][j], y_adj_train[i][j]))

Compute correlation to the gender vector:

In [36]:
corr_res = []

In [37]:
for i in range(len(models)):
    corr_res.append([])
    for j in range(n_folds):
        corr_res[i].append(correlation(X_adj_train[i][j], y_adj_train[i][j]))

In [38]:
for i in range(len(models)):
    for alpha in alphas:
        dims[labels[i]][f'LR{alpha}'] = []
        dims[labels[i]][f'Perc{alpha}'] = []
        dims[labels[i]][f'Corr{alpha}'] = []
        dims[labels[i]][f'All imp dims{alpha}'] = []
        for j in range(n_folds):
            num_imp_dims = len(X_adj_test[i][0].columns)*alpha//100
            lr_dims = [x[0] for x in lr_res[i][j][:num_imp_dims]]
            perc_dims = [x[0] for x in perceptron_res[i][j][:num_imp_dims]]
            corr_dims = [x[0] for x in corr_res[i][j][:num_imp_dims]]
            dims[labels[i]][f'LR{alpha}'].append(lr_dims)
            dims[labels[i]][f'Perc{alpha}'].append(perc_dims)
            dims[labels[i]][f'Corr{alpha}'].append(corr_dims)
            dims[labels[i]][f'All imp dims{alpha}'].append(list(set(lr_dims).intersection(perc_dims).intersection(corr_dims)))

In [39]:
dims[labels[0]]['LR1']

[[439, 250, 100, 464, 315],
 [439, 315, 260, 100, 275],
 [100, 250, 315, 260, 464],
 [100, 315, 464, 250, 260],
 [315, 100, 439, 429, 260]]

## Compute medians

In [40]:
medians = {}

In [41]:
for i in range(len(models)):
    medians[labels[i]] = {}
    for dim_group in dims[labels[i]].keys():
        medians[labels[i]][dim_group] = []
        n = len(dims[labels[i]][dim_group])
        for j in range(n_folds):
            dim_list = dims[labels[i]][dim_group][j] if n == n_folds else list(dims[labels[i]][dim_group])
            # Median of dimensions where feature vector is equal to 0
            median_0 = X_adj_train[i][j][y_adj_train[i][j] == 0][dim_list].median()
            # Median of dimensions where feature vector is equal to 1
            median_1 = X_adj_train[i][j][y_adj_train[i][j] == 1][dim_list].median()

            medians[labels[i]][dim_group].append({
                    '0': median_0,
                    '1': median_1
                })

We can compare the number of dimensions found by each test for each model in the first fold:

In [42]:
dim_lens = {}

for model in dims.keys():
    dim_lens[model] = {}
    for dim_group in dims[model].keys():
        dim_lens[model][dim_group] = len(dims[model][dim_group][0]) if dim_group != 'All dims' else \
                len(dims[model][dim_group])

In [43]:
pd.DataFrame(dim_lens)

,flau_small_c,flau_base_u,flau_base_c,flau_large_c,cam_base,xlm_large,xlm_base,bert_base_u,distilbert_base,bert_base_c
All dims,5,5,5,5,5,5,5,5,5,5
ANOVA,383,406,297,714,202,32,518,2,194,0
MI,378,664,439,751,544,545,564,424,524,468
All non ind,303,368,197,583,135,25,379,1,173,0
LR1,5,7,7,10,7,10,7,7,7,7
Perc1,5,7,7,10,7,10,7,7,7,7
Corr1,5,7,7,10,7,10,7,7,7,7
All imp dims1,1,0,0,2,0,0,0,1,0,0
LR5,25,38,38,51,38,51,38,38,38,38
Perc5,25,38,38,51,38,51,38,38,38,38


## Compute predictions

In [44]:
y_preds = {}

In [45]:
for i in range(len(models)):
    y_preds[labels[i]] = {}
    for dim_group in dims[labels[i]].keys():
        y_preds[labels[i]][dim_group] = []
        n = len(dims[labels[i]][dim_group])
        for j in range(n_folds):
            dim_list = dims[labels[i]][dim_group][j] if n == n_folds else dims[labels[i]][dim_group]
            
            # If MSE for sample 0 is lower than for sample 1, the label should be 0. So we need to convert False to 0.
            mae0 = X_adj_test[i][j][dim_list].apply(lambda x: mean_absolute_error(medians[labels[i]][dim_group][j]['0'], x), axis=1)
            mae1 = X_adj_test[i][j][dim_list].apply(lambda x: mean_absolute_error(medians[labels[i]][dim_group][j]['1'], x), axis=1)
            
            y_preds[labels[i]][dim_group].append((mae0 > mae1).apply(int))
    

## Compute accuracies

In [46]:
accs = {}

In [47]:
for i in range(len(models)):
    accs[labels[i]] = {}
    for dim_group in dims[labels[i]].keys():
        accs[labels[i]][dim_group] = []
        for j in range(n_folds):
            y_true = y_adj_test[i][j]
            y_pred = y_preds[labels[i]][dim_group][j]
            if any(y_pred):
                acc = accuracy_score(y_true, y_pred)
            else:
                acc = 0
            accs[labels[i]][dim_group].append(acc)

In [48]:
# Compute average accuracy between 5 folds
avg_accs = {}

In [49]:
for i in range(len(models)):
    avg_accs[labels[i]] = {}
    for dim_group in dims[labels[i]].keys():
        avg_accs[labels[i]][dim_group] = np.average(accs[labels[i]][dim_group])

In [50]:
accs_df = pd.DataFrame(avg_accs)
accs_df

,flau_small_c,flau_base_u,flau_base_c,flau_large_c,cam_base,xlm_large,xlm_base,bert_base_u,distilbert_base,bert_base_c
All dims,0.870351,0.571173,0.553397,0.865299,0.545536,0.539326,0.566486,0.521687,0.616208,0.534106
ANOVA,0.869819,0.583667,0.557647,0.864235,0.545536,0.524571,0.566486,0.509682,0.610530,0.000000
MI,0.866631,0.571844,0.556584,0.867955,0.545536,0.539326,0.566486,0.521687,0.608625,0.537916
All non ind,0.864772,0.586791,0.561101,0.867423,0.545536,0.522072,0.566486,0.306024,0.610530,0.000000
LR1,0.714127,0.573181,0.563763,0.732192,0.542189,0.549443,0.563987,0.521694,0.568643,0.528410
Perc1,0.734589,0.605980,0.577843,0.783462,0.555587,0.500060,0.558988,0.519277,0.629506,0.528464
Corr1,0.742032,0.613343,0.591126,0.794624,0.554178,0.544294,0.556489,0.512077,0.625750,0.522713
All imp dims1,0.494985,0.000000,0.000000,0.261355,0.217266,0.432099,0.111111,0.213859,0.000000,0.000000
LR5,0.823062,0.591920,0.581569,0.836073,0.547933,0.551701,0.571304,0.522892,0.644690,0.537951
Perc5,0.847772,0.635203,0.601746,0.866092,0.551771,0.534508,0.561548,0.520475,0.616173,0.543666


We can show what is the accuracy gain for each dimension test comparison to using all dimensions:

In [51]:
gains_df = (accs_df - accs_df.loc['All dims'])
gains_df

,flau_small_c,flau_base_u,flau_base_c,flau_large_c,cam_base,xlm_large,xlm_base,bert_base_u,distilbert_base,bert_base_c
All dims,0.000000e+00,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
ANOVA,-5.319149e-04,0.012494,4.250021e-03,-0.001064,0.000000,-0.014755,0.000000,-0.012004,-0.005678,-0.534106
MI,-3.719166e-03,0.000670,3.187251e-03,0.002656,0.000000,0.000000,0.000000,0.000000,-0.007583,0.003810
All non ind,-5.578749e-03,0.015617,7.703936e-03,0.002124,0.000000,-0.017254,0.000000,-0.215663,-0.005678,-0.534106
LR1,-1.562237e-01,0.002008,1.036634e-02,-0.133107,-0.003347,0.010117,-0.002499,0.000007,-0.047565,-0.005696
Perc1,-1.357612e-01,0.034806,2.444619e-02,-0.081837,0.010051,-0.039265,-0.007498,-0.002410,0.013297,-0.005642
Corr1,-1.283190e-01,0.042170,3.772922e-02,-0.070675,0.008642,0.004968,-0.009997,-0.009609,0.009542,-0.011393
All imp dims1,-3.753659e-01,-0.571173,-5.533971e-01,-0.603944,-0.328270,-0.107227,-0.455375,-0.307828,-0.616208,-0.534106
LR5,-4.728815e-02,0.020747,2.817171e-02,-0.029225,0.002397,0.012376,0.004818,0.001205,0.028482,0.003845
Perc5,-2.257813e-02,0.064029,4.834845e-02,0.000793,0.006235,-0.004818,-0.004938,-0.001212,-0.000036,0.009560


Here are the dimension groups and the accuracy gain this dimension group provides over all dimensions:

In [52]:
for i in range(len(models)):
    print(labels[i], gains_df.idxmax()[i], gains_df.loc[gains_df.idxmax()[i], labels[i]])

flau_small_c All imp dims50 0.020458944364386555
flau_base_u Perc5 0.06402940356744691
flau_base_c Perc10 0.04915056087705927
flau_large_c Perc25 0.020722429431211165
cam_base Perc1 0.01005128911225095
xlm_large LR5 0.012375790424571154
xlm_base LR10 0.009846431797651212
bert_base_u LR25 0.00121212121212122
distilbert_base All imp dims25 0.028517520215633363
bert_base_c All imp dims25 0.011428571428571344


In [53]:
for i in range(len(models)):
    print(labels[i], accs_df.idxmax()[i], accs_df.loc[accs_df.idxmax()[i], labels[i]])

flau_small_c All imp dims50 0.8908094572066345
flau_base_u Perc5 0.6352026596591813
flau_base_c Perc10 0.6025476109745416
flau_large_c Perc25 0.8860211635726596
cam_base Perc1 0.5555873004945326
xlm_large LR5 0.5517012947907258
xlm_base LR10 0.5763324299909666
bert_base_u LR25 0.522898868200073
distilbert_base All imp dims25 0.6447259658580413
bert_base_c All imp dims25 0.5455345911949685


For each fold find dimensions with the highest accuracy, and pick dimensions appeared in all 5 folds:

In [54]:
best_dims = {}

In [55]:
dim_cand_accs = {}

In [56]:
for i in range(len(labels)):
    bd = []
    for j in range(n_folds):
        best_dim_group = ''
        best_acc = 0
        for dim_group in accs[labels[i]].keys():
            if accs[labels[i]][dim_group][j] > best_acc:
                best_dim_group = dim_group
                best_acc = accs[labels[i]][dim_group][j]
        bd.extend(dims[labels[i]][best_dim_group][j])
    
    # Find dimensions that appeared within dimensions with the highest accuracy in all 5 folds
    d, c = np.unique(bd, return_counts=True)
    dim_cand = [x[0] for x in zip(d, c) if x[1] >= 5]
    best_dims[labels[i]] = dim_cand
    print(f'{labels[i]}: {len(dim_cand)} dimensions repeated in 5 folds')

    # For each fold build a prediction and calculate accuracy
    cand_accs = []
    for j in range(n_folds):
        med0_cand = X_adj_train[i][j][y_adj_train[i][j] == 0][dim_cand].median()
        med1_cand = X_adj_train[i][j][y_adj_train[i][j] == 1][dim_cand].median()
        
        mae0_cand = X_adj_test[i][j][dim_cand].apply(lambda x: mean_absolute_error(med0_cand, x), axis=1)
        mae1_cand = X_adj_test[i][j][dim_cand].apply(lambda x: mean_absolute_error(med1_cand, x), axis=1)
        
        y_pred_cand = (mae0_cand > mae1_cand).apply(int)
        if any(y_pred_cand):
            cand_accs.append(accuracy_score(y_true=y_adj_test[i][j], y_pred=y_pred_cand))
        else:
            cand_accs.append(0)
    dim_cand_accs[labels[i]] = cand_accs
    print(f'Average accuracy: {np.average(cand_accs)}')

flau_small_c: 77 dimensions repeated in 5 folds
Average accuracy: 0.8894828487468566
flau_base_u: 5 dimensions repeated in 5 folds
Average accuracy: 0.6229370719859849
flau_base_c: 5 dimensions repeated in 5 folds
Average accuracy: 0.5903273431098303
flau_large_c: 58 dimensions repeated in 5 folds
Average accuracy: 0.8791122743070272
cam_base: 0 dimensions repeated in 5 folds
Average accuracy: 0.0
xlm_large: 1 dimensions repeated in 5 folds
Average accuracy: 0.5367359229147847
xlm_base: 1 dimensions repeated in 5 folds
Average accuracy: 0.5785907859078591
bert_base_u: 0 dimensions repeated in 5 folds
Average accuracy: 0.0
distilbert_base: 0 dimensions repeated in 5 folds
Average accuracy: 0.0
bert_base_c: 0 dimensions repeated in 5 folds
Average accuracy: 0.0


In [60]:
with open('../Data/best_results/pos_adj_dims.pickle', 'wb') as f:
    pickle.dump(dims, f)

In [61]:
with open('../Data/best_results/pos_adj_accs.pickle', 'wb') as f:
    pickle.dump(accs, f)

In [62]:
with open('../Data/best_results/pos_adj_medians.pickle', 'wb') as f:
    pickle.dump(medians, f)

# Conclusion